# 위기탈출 헬 조선

## Data Processing 

### 1)Load Libraries

In [1]:
#import the libraries For data analysis 
import sys
import pandas as pd
import matplotlib
import numpy as np
import scipy as sp
import IPython
import sklearn

In [2]:
#Common Model Algorithms
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process

#Common Model Helpers
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics

#Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.tools.plotting import scatter_matrix

#### Korean Patch

한글 표시를 위해 실행

In [3]:
%matplotlib inline

import platform
path="c:/Windows/Fonts/malgun.ttf"
from matplotlib import font_manager, rc

if platform.system()=='Darwin':
    rc('font', family='AppleGothic')
elif platform.system() =='Windows':
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family = font_name)
else:
    print('Unknown system')

### 2)Load the Dataset for using this project

In [7]:
#Using traffic accident, tax, crime, population, cctv
traffic_data = pd.read_csv('./data/traffic/traffic-accident-2012_2017.csv', encoding='utf-8')
tax_data = pd.read_csv('./data/tax/tax_total.csv', encoding='utf-8')
#crime_2015_data = pd.read_csv('./data/crime/crime-city-2015_1.csv', encoding='utf-8')
crime_2016_data = pd.read_csv('./data/crime/crime-city-2016_1.csv', encoding='utf-8')
population_data = pd.read_csv('./data/population/population.csv', encoding='utf-8')
cctv_data = pd.read_csv('./data/cctv.csv', encoding='utf-8')

데이터를 체크해 본다

In [15]:
traffic_data.head()

,발생년,발생년월일시,주야,요일,사망자수,사상자수,발생지
0,2012,2012050901,야간,수,1,1,경남 거창군
1,2012,2012033013,주간,금,1,1,경기 용인시
2,2012,2012102620,야간,금,1,2,경북 김천시
3,2012,2012062814,주간,목,1,1,경기 이천시
4,2012,2012061804,야간,월,1,1,전남 광양시


교통사고 데이터에서 필요한 것은 발생년, 발생지 정도이므로 수정한다

In [26]:
tax_data.head()

,징수년,징수시도,징수시군구,징수분류,징수금액
0,2014,강원도,춘천시,취득세,66199922
1,2014,강원도,원주시,취득세,101985605
2,2014,강원도,강릉시,취득세,45786650
3,2014,강원도,동해시,취득세,18988361
4,2014,강원도,태백시,취득세,9402039


여기서는 정확하게 소득 수준의 분류를 위한 파악을 해야 하므로 징수분류를 다시 들여다 본다

In [35]:
tax_data[tax_data['징수시군구'].isin(['연수구'])]

,징수년,징수시도,징수시군구,징수분류,징수금액
1416,2014,인천,연수구,취득세,184481625
1426,2014,인천,연수구,주민세,907763
1436,2014,인천,연수구,지방소득세,50720024
1446,2015,인천,연수구,취득세,252978667
1456,2015,인천,연수구,주민세,1729233
1466,2015,인천,연수구,지방소득세,64302193
1476,2016,인천,연수구,취득세,210909544
1486,2016,인천,연수구,주민세,1917763
1496,2016,인천,연수구,지방소득세,101344375


징수분류는 취득세 주민세 지방소득세 이렇게 3가지로 나뉘게 된다

추 후에 필요한 년도와 필요한 시군구로 분류하여 총 합을 구한다

In [10]:
crime_2016_data.head()

,범죄대분류,범죄중분류,계,서울,부산,대구,인천,광주,대전,울산,...,경기 시흥,경기 파주,경남 양산,경기 이천,경기 구리,충남 서산,충북 제천,충남 논산,기타도시,도시이외
0,강력범죄,살인기수,356,46,29,17,20,9,16,7,...,8,3,3,1,1,2,1,3,30,51
1,강력범죄,살인미수등,558,100,43,12,28,8,9,15,...,7,2,1,0,5,3,3,0,67,49
2,강력범죄,강도,"1,149",260,137,51,88,47,35,33,...,9,4,7,2,1,5,2,1,72,46
3,강력범죄,강간,"5,155","1,129",314,197,347,170,171,112,...,57,38,31,26,30,10,14,13,367,366
4,강력범죄,유사강간,583,123,28,37,47,21,14,16,...,4,3,4,1,3,0,1,2,36,33


마찬가지로 범죄분류 또한 확인해본다

In [37]:
print(crime_2016_data['범죄대분류'])

0       강력범죄
1       강력범죄
2       강력범죄
3       강력범죄
4       강력범죄
5       강력범죄
6       강력범죄
7       강력범죄
8       절도범죄
9       폭력범죄
10      폭력범죄
11      폭력범죄
12      폭력범죄
13      폭력범죄
14      폭력범죄
15      폭력범죄
16      폭력범죄
17      지능범죄
18      지능범죄
19      지능범죄
20      지능범죄
21      지능범죄
22      지능범죄
23      지능범죄
24      지능범죄
25      지능범죄
26      풍속범죄
27      풍속범죄
28    특별경제범죄
29      마약범죄
30      보건범죄
31      환경범죄
32      교통범죄
33      노동범죄
34      안보범죄
35      선거범죄
36      병역범죄
37      기타범죄
Name: 범죄대분류, dtype: object


필요한 데이터는 강력범죄~폭력범죄 이므로 나머지 데이터 셋은 추후에 삭제한다

In [12]:
population_data.head()

,행정구역별,연도별,연령별,총인구,총인구_남자,총인구_여자,내국인_남자,내국인_여자,총인구_외국인,외국인_남자,외국인_여자,외국인비율
0,서울,2017,합계,9741871.0,4757642.0,4984229.0,4592393.0,4805551.0,343927.0,165249.0,178678.0,0.0353
1,서울,2017,0~4세,354174.0,181170.0,173004.0,177401.0,169310.0,7463.0,3769.0,3694.0,0.0211
2,서울,2017,5~9세,363637.0,186597.0,177040.0,184234.0,174832.0,4571.0,2363.0,2208.0,0.0126
3,서울,2017,10~14세,371556.0,191682.0,179874.0,190196.0,178551.0,2809.0,1486.0,1323.0,0.0076
4,서울,2017,15~19세,496075.0,251735.0,244340.0,247939.0,238520.0,9616.0,3796.0,5820.0,0.0194


In [38]:
population_data[population_data['행정구역별'].isin(['서울'])]

,행정구역별,연도별,연령별,총인구,총인구_남자,총인구_여자,내국인_남자,내국인_여자,총인구_외국인,외국인_남자,외국인_여자,외국인비율
0,서울,2017,합계,9741871.0,4757642.0,4984229.0,4592393.0,4805551.0,343927.0,165249.0,178678.0,0.0353
1,서울,2017,0~4세,354174.0,181170.0,173004.0,177401.0,169310.0,7463.0,3769.0,3694.0,0.0211
2,서울,2017,5~9세,363637.0,186597.0,177040.0,184234.0,174832.0,4571.0,2363.0,2208.0,0.0126
3,서울,2017,10~14세,371556.0,191682.0,179874.0,190196.0,178551.0,2809.0,1486.0,1323.0,0.0076
4,서울,2017,15~19세,496075.0,251735.0,244340.0,247939.0,238520.0,9616.0,3796.0,5820.0,0.0194
5,서울,2017,20~24세,699945.0,338539.0,361406.0,325097.0,336472.0,38376.0,13442.0,24934.0,0.0548
6,서울,2017,25~29세,777218.0,383505.0,393713.0,364581.0,370742.0,41895.0,18924.0,22971.0,0.0539
7,서울,2017,30~34세,766607.0,384052.0,382555.0,363965.0,363718.0,38924.0,20087.0,18837.0,0.0508
8,서울,2017,35~39세,806213.0,407731.0,398482.0,391846.0,385786.0,28581.0,15885.0,12696.0,0.0355
9,서울,2017,40~44세,756546.0,378503.0,378043.0,364563.0,367465.0,24518.0,13940.0,10578.0,0.0324


필요한 데이터는 합계 데이터만 필요하므로 

나머지 데이터 행은 삭제한다

In [14]:
cctv_data.head()

,city_name,district_name,CCTV
0,서울특별시,서울,44457
1,인천광역시,인천,8680
2,제주특별자치도,제주,2249
3,세종특별자치시,세종,1848
4,대전광역시,대전,6356


이미 한번 가공된 데이터 셋이다

### 3)'4C' on the Data

Correct, Convert, Clear, Completing

우선 우리가 분석할 데이터는 2016년의 데이터

현재 가지고 있는 데이터 중 2016년이 아닌 전체의 데이터도 있어서 2016년만 추출 하는 것이 필요

In [43]:
traffic_data.set_index('발생년')

,발생년월일시,주야,요일,사망자수,사상자수,발생지
발생년,,,,,,
2012,2012050901,야간,수,1,1,경남 거창군
2012,2012033013,주간,금,1,1,경기 용인시
2012,2012102620,야간,금,1,2,경북 김천시
2012,2012062814,주간,목,1,1,경기 이천시
2012,2012061804,야간,월,1,1,전남 광양시
2012,2012071508,주간,일,1,5,전남 광양시
2012,2012053104,야간,목,1,2,경기 하남시
2012,2012091714,주간,월,1,1,충남 논산시
2012,2012100102,야간,월,1,5,경남 밀양시


In [45]:
print(traffic_data[traffic_data['발생년'].isin(['2016'])])

        발생년      발생년월일시  주야 요일  사망자수  사상자수         발생지
19071  2016  2016122320  야간  금     1     1      경기 광주시
19072  2016  2016122517  주간  일     1     1      서울 금천구
19073  2016  2016122519  야간  일     1     1      충북 진천군
19074  2016  2016122610  주간  월     1     1      경북 경주시
19075  2016  2016122819  야간  수     1     1      경남 진주시
19076  2016  2016111207  주간  토     1     1      경남 거제시
19077  2016  2016110919  야간  수     1     1      경기 포천시
19078  2016  2016111005  야간  목     1     1       인천 서구
19079  2016  2016110711  주간  월     1     1      충남 예산군
19080  2016  2016110818  야간  화     1     1      전남 순천시
19081  2016  2016111302  야간  일     1     2      서울 용산구
19082  2016  2016111318  야간  일     1     1      전남 보성군
19083  2016  2016111915  주간  토     1     1      경기 가평군
19084  2016  2016111206  주간  토     1     1      경기 파주시
19085  2016  2016112700  야간  일     1     1      경북 칠곡군
19086  2016  2016111914  주간  토     1     1      서울 중랑구
19087  2016  2016112523  야간  금     1     1      경기 화성시
19088  201

In [50]:
key = np.arrange=(0,19070)
traffic_data.drop(key, axis=0)

KeyError: 'labels [(0, 19070)] not contained in axis'